## Final Features Model - Neural Net test using Fastai

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import datetime
import missingno as msno
import lightgbm as lgb
import xgboost as xgb
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, TimeSeriesSplit, train_test_split,StratifiedKFold
import gc
from statistics import mean
import pickle
# Any results you write to the current directory are saved as output.

In [2]:
from fastai.tabular import * 

### Setup Cross Validation
1. Divide Train set in subsets (Training set itself + Local Test set)
2. Define Validation Metric (in our case it is ROC-AUC)
3. Stop training when Validation metric stops improving
4. Take average of each fold's prediction for the Local Test set.

In [46]:
train_full = pd.read_pickle('data/train_feat.pkl')
test_full = pd.read_pickle('data/test_feat.pkl')

# Not using the below since the object type is all converted to category type already
# Label Encoding for categoricals
# for f in test_full.columns:
#     if train_full[f].dtype=='object' or test_full[f].dtype=='object': 
#         train_full[f] = train_full[f].fillna('unseen_before_label')
#         test_full[f]  = test_full[f].fillna('unseen_before_label')
#         lbl = preprocessing.LabelEncoder()
#         lbl.fit(list(train_full[f].values) + list(test_full[f].values))
#         train_full[f] = lbl.transform(list(train_full[f].values))
#         test_full[f] = lbl.transform(list(test_full[f].values)) 

# Fill NA's for numerics
train_full = train_full.fillna(-999)
test_full = test_full.fillna(-999)

In [33]:
with open('data/corr_feat.pkl', 'rb') as f:
    correlated_features = pickle.load(f)
    
rm_cols = [
    'TransactionID','TransactionDT', 
    'isFraud',
    'id_31'  # remove time dependent features - these cause model to overfit too much
#     'V300','V309','V111','V124','V106','V125','V315','V134','V102','V123','V316','V113',
#               'V136','V305','V110','V299','V289','V286','V318','V304','V116','V284','V293',
#               'V137','V295','V301','V104','V311','V115','V109','V119','V321','V114','V133','V122','V319',
#               'V105','V112','V118','V117','V121','V108','V135','V320','V303','V297','V120',
#               'V1','V14','V41','V65','V88', 'V89', 'V107', 'V68', 'V28', 'V27', 'V29', 'V241','V269',
#               'V240', 'V325', 'V138', 'V154', 'V153', 'V330', 'V142', 'V195', 'V302', 'V328', 'V327', 
#               'V198', 'V196', 'V155' # remove bunch of V features
] #+ correlated_features

# Final features
features_columns = [col for col in list(train_full.columns) if col not in rm_cols]

In [49]:
categoricals = []

for col in train_full.columns:
    if train_full[col].dtype == 'object':
        categoricals.append(col)

In [51]:
# pre-process using fastai
procs = [FillMissing, Categorify, Normalize]

dep_var = 'isFraud'

sample = train_full.sample(frac=0.2,random_state =123)
valid_idx = sample.index

In [68]:
# data = TabularDataBunch.from_df('data/', train_full, dep_var,valid_idx=valid_idx, procs=procs, cat_names=categoricals)

data = (TabularList.from_df(train_full, path='data/', cat_names=categoricals, procs=procs)
                        .split_by_idx(list(valid_idx))
                        .label_from_df(cols = dep_var)
                        .add_test(TabularList.from_df(test_full, cat_names=categoricals, procs=procs))
                        .databunch())

In [69]:
learn = tabular_learner(data, layers=[200,100], metrics=[accuracy])
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time


MemoryError: 

In [ ]:
predictions=learn.get_preds(DatasetType.Test)

In [50]:
sample_submission = pd.read_csv('data/sample_submission.csv', index_col='TransactionID')
sample_submission['isFraud'] = prediction
sample_submission.to_csv('data/xgboost_finalfeat.csv')

In [24]:
X.shape

(590540, 227)